# Running experiments from .yaml and saving results in .csv

In [1]:
import yaml
import csv
import torch
import numpy as np

In [2]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(128, 128)
        self.conv2 = GCNConv(128, 16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [3]:
from measures import Result


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.set_printoptions(precision=2)
np.set_printoptions(precision=2)

with open('experiments.yaml', 'r') as yaml_file:
    experiments = yaml.safe_load(yaml_file)

with open('results.csv', 'w', newline='') as csv_file:
    fieldnames = ['Dataset', 'AP', 'AF']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for experiment in experiments['experiments']:
        print(experiment['name'], experiment['directory'])
        for i in range(10):
            csbm = torch.load(experiment['directory'] + experiment['name'] + f'{i}.pt')
            result = Result(GCN().to(device), csbm)
            result.learn()
            ap = result.get_average_accuracy()
            af = 0 if experiment['name'] == 'zero_' else result.get_average_forgetting_measure()
            writer.writerow({'Dataset': f'{experiment["name"]}', 'AP': f'{ap:.2f}', 'AF': f'{af:.2f}'})
            print(f'AP: {ap:.2f}'.ljust(10) + f' AF: {af:.2f}')
        print()

cuda
base_ ./csbm_base/
AP: 0.28   AF: 0.36
AP: 0.30   AF: 0.27
AP: 0.30   AF: 0.27
AP: 0.20   AF: 0.40
AP: 0.24   AF: 0.42
AP: 0.22   AF: 0.31
AP: 0.30   AF: 0.38
AP: 0.36   AF: 0.27
AP: 0.28   AF: 0.31
AP: 0.28   AF: 0.27

zero_ ./csbm_zero/
AP: 0.36   AF: 0.00
AP: 0.34   AF: 0.00
AP: 0.24   AF: 0.00
AP: 0.42   AF: 0.00
AP: 0.36   AF: 0.00
AP: 0.30   AF: 0.00
AP: 0.46   AF: 0.00
AP: 0.26   AF: 0.00
AP: 0.32   AF: 0.00
AP: 0.42   AF: 0.00

feat_ ./csbm_feat/
AP: 0.10   AF: 0.44
AP: 0.24   AF: 0.29
AP: 0.16   AF: 0.33
AP: 0.10   AF: 0.47
AP: 0.16   AF: 0.38
AP: 0.16   AF: 0.40
AP: 0.10   AF: 0.49
AP: 0.14   AF: 0.44
AP: 0.20   AF: 0.29
AP: 0.16   AF: 0.36

struct_ ./csbm_struct/
AP: 0.20   AF: 0.38
AP: 0.28   AF: 0.22
AP: 0.30   AF: 0.13
AP: 0.16   AF: 0.36
AP: 0.22   AF: 0.36
AP: 0.22   AF: 0.27
AP: 0.26   AF: 0.24
AP: 0.28   AF: 0.29
AP: 0.28   AF: 0.18
AP: 0.28   AF: 0.27

hom_ ./csbm_hom/
AP: 0.34   AF: 0.29
AP: 0.34   AF: 0.31
AP: 0.28   AF: 0.29
AP: 0.34   AF: 0.18
AP: 0.26   AF:

In [4]:
import pandas as pd

performance = pd.read_csv('results.csv')
print(performance.groupby('Dataset').mean())

C:\Users\Produktivität\AppData\Local\Temp\ipykernel_9184\3817551194.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


            AP     AF
Dataset              
base_    0.276  0.326
class_   0.274  0.409
feat_    0.152  0.389
hom_     0.310  0.287
struct_  0.248  0.270
zero_    0.348  0.000
